# Example Pyllelic Use-Case Notebook

## Background

This notebook illustrates the import and use of `pyllelic` in a jupyter environment.

See https://github.com/Paradoxdruid/pyllelic for further details.

## Set-up

In [ ]:
import pyllelic

In [ ]:
# set up your disk location:
# base_path should be the directory we'll do our work in
# make a sub-directory under base_path with a folder named "test"
# and put the .bam and .bai files in "test"

pyllelic.set_up_env_variables(
    base_path="/Users/abonham/documents/test_allelic/",
    prom_file="TERT-promoter-genomic-sequence.txt",
    prom_start="1293000",
    prom_end="1296000",
    chrom="5",
)

## Main Parsing Functions

In [ ]:
files_set = pyllelic.make_list_of_bam_files()  # finds bam files

In [ ]:
# Uncomment for debugging:
# files_set

In [ ]:
# index bam and creates bam_output folders/files
positions = pyllelic.index_and_fetch(files_set)

In [ ]:
pyllelic.genome_parsing()

In [ ]:
cell_types = pyllelic.extract_cell_types(files_set)

In [ ]:
# Uncomment for debugging
# cell_types

In [ ]:
# Set filename to whatever you want
df_list = pyllelic.run_quma_and_compile_list_of_df(cell_types, "tester2.xlsx")

In [ ]:
# Uncomment for debugging
# df_list.keys()

In [ ]:
means = pyllelic.process_means(df_list, positions, files_set)

In [ ]:
# Uncomment for debugging
# means

In [ ]:
modes = pyllelic.process_modes(df_list, positions, files_set)

In [ ]:
# Uncomment for debugging
# modes

In [ ]:
diff = pyllelic.find_diffs(means, modes)

In [ ]:
# Uncomment for debugging
# diff

## Write Output to excel files

In [ ]:
# Set the filename to whatever you want
pyllelic.write_means_modes_diffs(means, modes, diff, "Test_Output")